In [1]:
import re

import pandas
import s3fs
import sqlalchemy

import cpas_reports

## Collating CPAS Reports

The CPAS web portal has a few pre-canned reports. We have copied those,
and placed them in `cpas_reports.py`.

The following collates those reports for all the grant years,
and uploads them to S3 as parquet files.

These should be considered views onto the underlying tables,
but are a convenient entry point into the CPAS database.

In [2]:
def combine_reports(callable, min_year, max_year):
    combined = pandas.DataFrame()
    for year in range(min_year, max_year + 1):
        try:
            df = callable(year)
            combined = pandas.concat([combined, df], axis=0, sort=False)
        except sqlalchemy.exc.DatabaseError as e:
            print(f"Failed to load year {year} with message {str(e)[:89]}")
    return combined.reset_index(drop=True)

In [3]:
MIN_YEAR = 30
MAX_YEAR = 46

pep_info = combine_reports(cpas_reports.get_pep_info, MIN_YEAR, MAX_YEAR)
application_info = combine_reports(cpas_reports.get_application_info, MIN_YEAR, MAX_YEAR)
application_list = combine_reports(cpas_reports.app_list, MIN_YEAR, MAX_YEAR)
grnt_gpr = combine_reports(cpas_reports.get_grnt_gpr, MIN_YEAR, MAX_YEAR)

Failed to load year 30 with message (cx_Oracle.DatabaseError) ORA-00942: table or view does not exist [SQL: "\n \tSELECT \n\t
Failed to load year 31 with message (cx_Oracle.DatabaseError) ORA-00942: table or view does not exist [SQL: "\n \tSELECT \n\t
Failed to load year 30 with message (cx_Oracle.DatabaseError) ORA-00942: table or view does not exist [SQL: '\n    SELECT\n  
Failed to load year 31 with message (cx_Oracle.DatabaseError) ORA-00942: table or view does not exist [SQL: '\n    SELECT\n  
Failed to load year 32 with message (cx_Oracle.DatabaseError) ORA-00904: "KNWN_SITE_LOC_YN": invalid identifier [SQL: '\n    
Failed to load year 33 with message (cx_Oracle.DatabaseError) ORA-00904: "GRNT_L_PROJ_LOC_RSN": invalid identifier [SQL: '\n 
Failed to load year 34 with message (cx_Oracle.DatabaseError) ORA-00904: "GRNT_L_PROJ_LOC_RSN": invalid identifier [SQL: '\n 
Failed to load year 35 with message (cx_Oracle.DatabaseError) ORA-00904: "PRPSD_CONTRACT_CNT": invalid identifier [SQL

In [4]:
BUCKET = "s3://hcid-cdbg-project-ita-data"
pep_info.to_parquet(f"{BUCKET}/pep_info.parquet")
application_info.to_parquet(f"{BUCKET}/application_info.parquet")
application_list.to_parquet(f"{BUCKET}/application_list.parquet")
grnt_gpr.to_parquet(f"{BUCKET}/grant_gpr.parquet")

## Where is the data in CPAS?

The block grant data seems to be mostly organized by grant year in schemas/users titled "`GRNT{year}`".
Each of those schemas has a large number of tables, which are similar, but not identical from year to year.
Let's get some descriptive statistics of all of the tables.

In [5]:
engine = cpas_reports.engine
schemas = sqlalchemy.inspect(engine).get_schema_names()
# Get all the schemas starting with `grnt`, indicating a single grant year.
schemas = [s for s in schemas if re.match("grnt\d+", s)]
schemas

['grnt30',
 'grnt31',
 'grnt32',
 'grnt33',
 'grnt34',
 'grnt35',
 'grnt36',
 'grnt37',
 'grnt38',
 'grnt39',
 'grnt40',
 'grnt41',
 'grnt42',
 'grnt43',
 'grnt44',
 'grnt45',
 'grnt46']

In [6]:
# Collect all the tables in each schema, as well as a counter dictionary
# to see how many times each table name appears across the schemas
counts = {}
data = {}
for schema in schemas:
    tables = engine.table_names(schema=schema)
    data[schema] = tables
    for t in tables:
        counts[t] = counts.get(t, 0) + 1

In [7]:
# Create a meta-table describing whether a given table appears in a schema
all_tables = set(counts.keys())
tables_df = pandas.DataFrame({key: [t in data[key] for t in all_tables] for key in data}, index=all_tables)
tables_df = tables_df.assign(count=pandas.Series(counts)).sort_values("count", ascending=False)

In [8]:
# Visualize which tables are in common in a given schema:
tables_df.style.applymap(lambda x: "color: green" if x == True else "color: red" if x == False else "")

In [9]:
# All the tables which exist in every grant year since 39
# This year is somewhat arbitrarily chosen,
# but it does correspond to the most commonly used tables in recent years.
common_tables = tables_df[tables_df[schemas[9:]].all(axis=1)].index
common_tables

Index(['grnt_src_fnd', 'grnt_l_prjct_prty', 'grnt_bud', 'grnt_cptl_plan',
       'grnt_cntct', 'grnt_l_note_typ', 'grnt_l_aplc_elig', 'grnt_hstrcl_rvw',
       'grnt_l_stat', 'grnt_stat', 'grnt_cptl_plan_dtl', 'grnt_aplctn_prog',
       'grnt_l_recom_aprv', 'grnt_l_aplc_readiness', 'grnt_prjct_fnd',
       'grnt_prr_yr_cdbg', 'grnt_prjct_loctn', 'grnt_form', 'grnt_aplctn',
       'grnt_l_fnd', 'grnt_l_attchmnt', 'grnt_chk_list', 'grnt_accmplsh_dtl',
       'grnt_l_accmplsh', 'grnt_case_note', 'grnt_bud_expns', 'grnt_hud_cd',
       'grnt_prjct_accmplsh', 'serv_area_census_tract', 'grnt_cptl_typ',
       'grnt_bud_gnrl_dtl', 'grnt_loctn_typ', 'grnt_hud_objctive',
       'grnt_l_expns', 'grnt_aplctn_note', 'grnt_aplctn_attchmnt',
       'grnt_pep_cncl_dist', 'grnt_l_loc', 'grnt_l_pep_typ',
       'grnt_pep_accmplsh', 'grnt_l_rec_color', 'grnt_l_send_out_mthd',
       'grnt_pep_lvrg_resrc', 'grnt_pep_prjctd_reimb',
       'grnt_l_proj_prity_mayor', 'grnt_pep_note', 'grnt_l_objctive',
    

In [10]:
def combine_grant_years(t, engine):
    combined = pandas.DataFrame()
    for year in range(MIN_YEAR, MAX_YEAR+1):
        try:
            df = pandas.read_sql(f"SELECT * FROM grnt{year}.{t}", engine)
            df = df.assign(year=year)
            combined = pandas.concat([combined, df], axis=0, sort=False)
        except sqlalchemy.exc.DatabaseError as e:
            print(f"Failed to load year {year} with message {str(e)[:89]}")
    return combined.reset_index(drop=True)

In [11]:
# Combine the common tables across grant years
database = {}
for t in common_tables:
    combined = combine_grant_years(t, engine)
    database[t] = combined
    print(f"Loaded {t}, with {len(combined)} records")

Loaded grnt_src_fnd, with 3339 records
Loaded grnt_l_prjct_prty, with 135 records
Loaded grnt_bud, with 3276 records
Loaded grnt_cptl_plan, with 574 records
Loaded grnt_cntct, with 6130 records
Loaded grnt_l_note_typ, with 289 records
Loaded grnt_l_aplc_elig, with 68 records
Loaded grnt_hstrcl_rvw, with 76 records
Loaded grnt_l_stat, with 136 records
Loaded grnt_stat, with 3095 records
Loaded grnt_cptl_plan_dtl, with 12774 records
Loaded grnt_aplctn_prog, with 17761 records
Loaded grnt_l_recom_aprv, with 34 records
Loaded grnt_l_aplc_readiness, with 68 records
Loaded grnt_prjct_fnd, with 4729 records
Loaded grnt_prr_yr_cdbg, with 1867 records
Loaded grnt_prjct_loctn, with 4262 records
Loaded grnt_form, with 116 records
Loaded grnt_aplctn, with 3098 records
Loaded grnt_l_fnd, with 83 records
Loaded grnt_l_attchmnt, with 97 records
Loaded grnt_chk_list, with 2154 records
Loaded grnt_accmplsh_dtl, with 8824 records
Loaded grnt_l_accmplsh, with 272 records
Loaded grnt_case_note, with 4466 

In [12]:
# Drop tables if they start with "grnt_l_", as HCID indicates that those are not important,
# as well as tables that have fewer than 100 records, hopefully indicating that they
# are less-used
relevant_tables = {k for k,v in database.items() if not k.startswith("grnt_l_") and len(v) > 100}
database = {k: v for k,v in database.items() if k in relevant_tables}

In [13]:
# Create a sqlite database with the collated grant tables.

cpas = sqlalchemy.create_engine("sqlite:///cpas.sqlite")

for key, table in database.items():
    print(f"Writing {key}")
    # Attempt to fix broken datetimes
    coerce_dt = [
        c for c in table.columns
        if pandas.core.indexes.accessors.is_datetime_arraylike(table[c])
    ]
    table = table.assign(**{c: pandas.to_datetime(table[c].fillna(pandas.NaT), errors="coerce") for c in coerce_dt})
    # Write to SQLite
    table.to_sql(key, cpas, index=False, if_exists="replace")

Writing grnt_src_fnd
Writing grnt_bud
Writing grnt_cptl_plan
Writing grnt_cntct
Writing grnt_stat
Writing grnt_cptl_plan_dtl
Writing grnt_aplctn_prog
Writing grnt_prjct_fnd
Writing grnt_prr_yr_cdbg
Writing grnt_prjct_loctn
Writing grnt_form
Writing grnt_aplctn
Writing grnt_chk_list
Writing grnt_accmplsh_dtl
Writing grnt_case_note
Writing grnt_bud_expns
Writing grnt_hud_cd
Writing grnt_prjct_accmplsh
Writing serv_area_census_tract
Writing grnt_cptl_typ
Writing grnt_bud_gnrl_dtl
Writing grnt_hud_objctive
Writing grnt_aplctn_note
Writing grnt_aplctn_attchmnt
Writing grnt_pep_cncl_dist
Writing grnt_pep_accmplsh
Writing grnt_pep_lvrg_resrc
Writing grnt_pep_prjctd_reimb
Writing grnt_pep_note
Writing grnt_pep
Writing grnt_pps_accmplsh_dtl
Writing grnt_pps_accmplsh
Writing grnt_prpsd_proj
Writing grnt_pps_addr
Writing grnt_cbdo
Writing grnt_gpr_job_catg
Writing grnt_gpr_race
Writing grnt_gpr_census_tract
Writing grnt_gpr_duns
Writing grnt_gpr
Writing grnt_cbdo_doc
Writing grnt_gpr_incm_lvl
Wri

In [14]:
fs = s3fs.S3FileSystem()
fs.upload("./cpas.sqlite", f"{BUCKET}/cpas.sqlite")